In [4]:
import FocusStack
from glob import glob
from os.path import basename
import numpy as np
import cv2

In [15]:
bilder = glob('../../bilderserien/TPKarton/F*/camera04.jpg')
bilderDict: dict[str, list] = {}
for b in bilder:
    n = basename(b)
    if n not in bilderDict:
        bilderDict[n] = []
    bilderDict[n].append(cv2.imread(b))
images = bilderDict['camera04.jpg']

In [16]:
def findHomography(image_1_kp, image_2_kp, matches):
    image_1_points = np.zeros((len(matches), 1, 2), dtype=np.float32)
    image_2_points = np.zeros((len(matches), 1, 2), dtype=np.float32)
    for i in range(0, len(matches)):
        image_1_points[i] = image_1_kp[matches[i].queryIdx].pt
        image_2_points[i] = image_2_kp[matches[i].trainIdx].pt
    homography, mask = cv2.findHomography(
        image_1_points, image_2_points, cv2.RANSAC, ransacReprojThreshold=2.0)
    return homography

In [17]:
#   SIFT generally produces better results, but it is not FOSS, so chose the feature detector
#   that suits the needs of your project.  ORB does OK
use_sift = True

outimages = []

if use_sift:
    detector = cv2.xfeatures2d.SIFT_create()
else:
    detector = cv2.ORB_create(1000)

#   We assume that image 0 is the "base" image and align everything to it
print("Detecting features of base image")
outimages.append(images[0])
image1gray = cv2.cvtColor(images[0], cv2.COLOR_BGR2GRAY)
image_1_kp, image_1_desc = detector.detectAndCompute(image1gray, None)

for i in range(1, len(images)):
    print("Aligning image {}".format(i))
    image_i_kp, image_i_desc = detector.detectAndCompute(images[i], None)

    if use_sift:
        bf = cv2.BFMatcher()
        # This returns the top two matches for each feature point (list of list)
        pairMatches = bf.knnMatch(image_i_desc, image_1_desc, k=2)
        rawMatches = []
        for m, n in pairMatches:
            if m.distance < 0.7*n.distance:
                rawMatches.append(m)
    else:
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
        rawMatches = bf.match(image_i_desc, image_1_desc)

    sortMatches = sorted(rawMatches, key=lambda x: x.distance)
    matches = sortMatches[0:128]

    hom = findHomography(image_i_kp, image_1_kp, matches)
    print(hom)

Detecting features of base image
Aligning image 1
[[ 9.97317829e-01 -6.32937159e-04  7.94036481e+00]
 [ 3.38095344e-04  9.95854551e-01  5.56122848e+00]
 [ 2.97922399e-07 -2.65088026e-07  1.00000000e+00]]
Aligning image 2
[[ 9.82039084e-01 -3.98119500e-03  4.70981264e+01]
 [ 1.15171300e-03  9.72764470e-01  3.08505587e+01]
 [ 1.18550462e-06 -2.47418640e-06  1.00000000e+00]]
Aligning image 3
[[ 9.71762183e-01  9.33341268e-04  6.88607696e+01]
 [ 1.44605095e-03  9.59129863e-01  4.87045817e+01]
 [ 1.18294102e-06 -9.15289304e-07  1.00000000e+00]]
Aligning image 4
[[9.74443981e-01 2.60323630e-03 6.01448402e+01]
 [3.24399286e-04 9.74096265e-01 3.48434163e+01]
 [2.36550360e-07 9.91366661e-07 1.00000000e+00]]
Aligning image 5
[[ 9.32804649e-01 -7.75767449e-04  1.30657374e+02]
 [-7.30215097e-03  9.40267950e-01  7.90722699e+01]
 [-4.43913948e-06 -1.04377364e-06  1.00000000e+00]]


Detecting features of base image
Aligning image 1
[[ 9.85134338e-01 -1.94441602e-03  3.09039706e+01]
 [-7.46280738e-04  9.86053622e-01  1.86755548e+01]
 [-6.55692138e-07 -4.01536734e-07  1.00000000e+00]]
Aligning image 2
[[ 9.73494911e-01 -1.78204697e-03  5.53319679e+01]
 [-1.68667652e-03  9.76122090e-01  3.04705512e+01]
 [-1.30117625e-06 -1.43300602e-07  1.00000000e+00]]
Aligning image 3
[[ 9.70064949e-01 -3.20645210e-03  8.33205248e+01]
 [ 2.53368857e-03  9.63822237e-01  4.46591145e+01]
 [ 2.45135919e-06 -1.43876694e-06  1.00000000e+00]]
Aligning image 4
[[ 9.67711750e-01  3.94349359e-03  6.96564954e+01]
 [-1.73342747e-03  9.73736433e-01  3.71827876e+01]
 [-1.28694921e-06  2.17531778e-06  1.00000000e+00]]
Aligning image 5
[[ 9.50880158e-01 -2.13785905e-03  1.13850532e+02]
 [ 6.32479163e-04  9.51894191e-01  5.78017502e+01]
 [ 8.98913831e-08 -1.10454699e-06  1.00000000e+00]]

In [7]:
for bild in bilderDict:
    merged = FocusStack.focus_stack(bilderDict[bild])
    cv2.imwrite(f'../../bilderserien/Moai/SF30-60/{bild}', merged)

[ WARN:0@103.424] global shadow_sift.hpp:13 SIFT_create DEPRECATED: cv.xfeatures2d.SIFT_create() is deprecated due SIFT tranfer to the main repository. https://github.com/opencv/opencv/issues/16736


Detecting features of base image
Aligning image 1
Aligning image 2
Computing the laplacian of the blurred images
Lap 0
Lap 1
Lap 2
Shape of array of laplacians = (3, 2592, 4608)
Detecting features of base image
Aligning image 1
Aligning image 2
Computing the laplacian of the blurred images
Lap 0
Lap 1
Lap 2
Shape of array of laplacians = (3, 2592, 4608)
Detecting features of base image
Aligning image 1
Aligning image 2
Computing the laplacian of the blurred images
Lap 0
Lap 1
Lap 2
Shape of array of laplacians = (3, 2592, 4608)
Detecting features of base image
Aligning image 1
Aligning image 2
Computing the laplacian of the blurred images
Lap 0
Lap 1
Lap 2
Shape of array of laplacians = (3, 2592, 4608)
Detecting features of base image
Aligning image 1
Aligning image 2
Computing the laplacian of the blurred images
Lap 0
Lap 1
Lap 2
Shape of array of laplacians = (3, 2592, 4608)
Detecting features of base image
Aligning image 1
Aligning image 2
Computing the laplacian of the blurred imag